In [16]:
from google.colab import files
files.upload()  # Upload kaggle.json here

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"admin107","key":"7684edd75739ad8308d863055a7a6415"}'}

In [17]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [18]:
!kaggle datasets download -d jocelyndumlao/textbook-dataset-from-nctb
!unzip textbook-dataset-from-nctb.zip

Dataset URL: https://www.kaggle.com/datasets/jocelyndumlao/textbook-dataset-from-nctb
License(s): CC0-1.0
  0% 0.00/205k [00:00<?, ?B/s]
100% 205k/205k [00:00<00:00, 470MB/s]
Archive:  textbook-dataset-from-nctb.zip
  inflating: Textbook Dataset from NCTB/Textbook Dataset from NCTB.csv  


In [19]:
import pandas as pd

df = pd.read_csv("/content/Textbook Dataset from NCTB/Textbook Dataset from NCTB.csv")
print(df.head())

                                             Passage  \
0  খাদ্যের উপাদান ছয়টি।  সেগুলো হলো: শর্করা, আমি...   
1  খাদ্যের উপাদান ছয়টি।  সেগুলো হলো: শর্করা, আমি...   
2  খাদ্যের উপাদান ছয়টি।  সেগুলো হলো: শর্করা, আমি...   
3  খাদ্যের উপাদান ছয়টি।  সেগুলো হলো: শর্করা, আমি...   
4  খাদ্যের উপাদান ছয়টি।  সেগুলো হলো: শর্করা, আমি...   

                           Question   AnsText  
0          কোনটি দেহ পরিপোষক খাদ্য?      আমিষ  
1         কোনটি শক্তি উৎপাদক খাদ্য?     স্নেহ  
2            কোনটি দেহ গঠনের খাদ্য?    শর্করা  
3   কোনটি দেহ সংরক্ষক খাদ্য উপাদান?   ভিটামিন  
4  কোনটি রোগ প্রতিরোধে সাহায্য করে?  খনিজ লবণ  


In [20]:
from datasets import Dataset

# Rename columns if needed
df = df.rename(columns={"Passage": "context", "Question": "question", "AnsText": "answer"})

# Convert context and answer columns to string type
df['context'] = df['context'].astype(str)
df['answer'] = df['answer'].astype(str)

# Add answer_start (optional: use string matching)
df["answer_start"] = df.apply(lambda row: row["context"].find(row["answer"]), axis=1)

# Filter out rows where answer isn't found
df = df[df["answer_start"] != -1]

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)
print(dataset[0])

{'context': 'খাদ্যের উপাদান ছয়টি।  সেগুলো হলো: শর্করা, আমিষ, স্নেহ, ভিটামিন, খনিজ লবণ এবং পানি। এগুলোর মধ্যে শর্করা, আমিষ ও স্নেহ পদার্থ (বা ফ্যাট) দেহ পরিপোষক খাদ্য। খাদ্যের স্নেহ এবং শর্করাকে বলা হয় শক্তি উৎপাদক খাদ্য এবং আমিষ যুক্ত খাদ্যকে বলা হয় দেহ গঠনের খাদ্য। ভিটামিন, খনিজ লবণ ও পানি দেহ সংরক্ষক খাদ্য উপাদান, যেগুলো দেহের রোগ প্রতিরোধে সাহায্য করে।', 'question': 'কোনটি দেহ পরিপোষক খাদ্য?', 'answer': 'আমিষ', 'answer_start': 43, '__index_level_0__': 0}


In [21]:
from datasets import Dataset

# Rename columns if needed
df = df.rename(columns={"passage": "context", "question": "question", "answer": "answer"})

# Add answer_start (position of answer in context)
df["answer_start"] = df.apply(lambda row: row["context"].find(row["answer"]), axis=1)

# Filter out rows where answer isn't found
df = df[df["answer_start"] != -1]

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [22]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

model_name = "xlm-roberta-base"  # You can also try "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForQuestionAnswering were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
def preprocess(example):
    inputs = tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        padding="max_length",
        max_length=512,
        return_offsets_mapping=True  # Add this to get character to token mapping
    )
    start_char = example["answer_start"]
    end_char = start_char + len(example["answer"])

    # Find the start and end token indices using offset_mapping
    start_token = inputs.char_to_token(start_char)
    end_token = inputs.char_to_token(end_char - 1)

    # Handle cases where the answer is not within the tokenized context
    if start_token is None:
      # If the answer is not found in the tokenized context, set positions to 0
      start_token = 0
    if end_token is None:
      # If the answer is not found in the tokenized context, set positions to 0
      end_token = 0

    inputs["start_positions"] = start_token
    inputs["end_positions"] = end_token
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=False, remove_columns=dataset.column_names)

Map:   0%|          | 0/1023 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer, AutoTokenizer, AutoModelForQuestionAnswering
from datasets import Dataset
import pandas as pd

# Load the dataset
df = pd.read_csv("/content/Textbook Dataset from NCTB/Textbook Dataset from NCTB.csv")

# Rename columns
df = df.rename(columns={"Passage": "context", "Question": "question", "AnsText": "answer"})

# Ensure string types
df["context"] = df["context"].astype(str)
df["answer"] = df["answer"].astype(str)

# Add answer_start
df["answer_start"] = df.apply(lambda row: row["context"].find(row["answer"]), axis=1)

# Filter out rows with missing answers
df = df[df["answer_start"] != -1]

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Load tokenizer and model
model_name = "ai4bharat/indic-bert"  # Lighter than xlm-roberta-base
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

# Preprocessing with offset mapping
def preprocess(example):
    encoding = tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        padding="max_length",
        max_length=384,  # Increased for better span coverage
        return_offsets_mapping=True
    )

    start_char = example["answer_start"]
    end_char = start_char + len(example["answer"])
    offsets = encoding["offset_mapping"]

    start_token = end_token = None
    for idx, (start, end) in enumerate(offsets):
        if start <= start_char < end:
            start_token = idx
        if start < end_char <= end:
            end_token = idx
            break

    if start_token is None:
        start_token = 0
    if end_token is None:
        end_token = 0

    encoding["start_positions"] = start_token
    encoding["end_positions"] = end_token
    encoding.pop("offset_mapping")  # Remove for training

    return dict(encoding) # Return as a dictionary

# Tokenize dataset
tokenized_dataset = dataset.map(preprocess, batched=False, remove_columns=dataset.column_names)

# Split into train and eval sets
train_test_dict = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_dict["train"]
eval_dataset = train_test_dict["test"]

# Training arguments (optimized for Colab)
training_args = TrainingArguments(
    output_dir="bangla-qa-model",
    eval_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=4,  # Simulates larger batch
    report_to="none"
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
)

# Train and save
trainer.train()
trainer.save_model("bangla-qa-model")

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/135M [00:00<?, ?B/s]

Some weights of AlbertForQuestionAnswering were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1023 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/135M [00:00<?, ?B/s]

/tmp/ipython-input-4215176681.py:85: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss


In [ ]:
import gradio as gr
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="bangla-qa-model", tokenizer="bangla-qa-model")

def answer_question(passage, question):
    result = qa_pipeline(question=question, context=passage)
    return result["answer"]

gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Textbox(lines=10, label="📖 Bangla Passage"),
        gr.Textbox(lines=2, label="❓ Question")
    ],
    outputs=gr.Textbox(label="✅ Answer"),
    title="Bangla QA System",
    description="Ask questions from Bangla textbook passages."
).launch(share=True)